In [53]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
cal = pd.read_csv('C:/Users/DEBASHIS/Desktop/E-2-E Calory count prediction using XGboost/notebooks/data/calories.csv')
exc = pd.read_csv('C:/Users/DEBASHIS/Desktop/E-2-E Calory count prediction using XGboost/notebooks/data/exercise.csv')

In [19]:
df = cal.merge(exc, on='User_ID')
df.head()

,User_ID,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,14733363,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,14861698,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,11179863,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,16180408,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,17771927,35.0,female,27,154.0,58.0,10.0,81.0,39.8


In [20]:
df.drop(['User_ID'], axis=1, inplace=True)
df.head()


,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,231.0,male,68,190.0,94.0,29.0,105.0,40.8
1,66.0,female,20,166.0,60.0,14.0,94.0,40.3
2,26.0,male,69,179.0,79.0,5.0,88.0,38.7
3,71.0,female,34,179.0,71.0,13.0,100.0,40.5
4,35.0,female,27,154.0,58.0,10.0,81.0,39.8


In [22]:
encoder = LabelEncoder()
df['Gender'] = encoder.fit_transform(df['Gender'])

In [27]:
df.head()

,Calories,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,231.0,1,68,190.0,94.0,29.0,105.0,40.8
1,66.0,0,20,166.0,60.0,14.0,94.0,40.3
2,26.0,1,69,179.0,79.0,5.0,88.0,38.7
3,71.0,0,34,179.0,71.0,13.0,100.0,40.5
4,35.0,0,27,154.0,58.0,10.0,81.0,39.8


In [34]:
y = df['Calories']
X = df.drop(['Calories'], axis=1)

In [35]:
X.dtypes

Gender          int32
Age             int64
Height        float64
Weight        float64
Duration      float64
Heart_Rate    float64
Body_Temp     float64
dtype: object

In [41]:
columns_to_scale = X.describe().columns
columns_to_scale = columns_to_scale.drop(['Gender'])
columns_to_scale

Index(['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp'], dtype='object')

In [42]:

standard_scaler = StandardScaler()
X[columns_to_scale] = standard_scaler.fit_transform(X[columns_to_scale])

In [44]:
X.head()

,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,1,1.484726,1.089582,1.265909,1.619127,0.989404,0.994023
1,0,-1.342179,-0.593726,-0.995458,-0.183990,-0.158461,0.352342
2,1,1.543620,0.318066,0.268247,-1.265861,-0.784569,-1.701035
3,0,-0.517665,0.318066,-0.263839,-0.304198,0.467647,0.609015
4,0,-0.929922,-1.435380,-1.128480,-0.664821,-1.515029,-0.289338


In [45]:
standard_scaler.var_

array([288.31014929, 203.28025098, 226.05590218,  69.20453031,
        91.83405652,   0.60715879])

In [51]:
age_pipeline = Pipeline(steps=[
    ('scaler', LabelEncoder())
])

In [52]:
num_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [55]:
pre_processor = ColumnTransformer(transformers=[
    ('age', age_pipeline, ['Age']),
    ('num', num_pipeline, columns_to_scale)
])